In [9]:
import pandas as pd
import os

In [2]:
# File paths
survey_path = "./data_cleaned_capped/survey.csv"
ridership_path = "./data_cleaned_capped/ridership.csv"
arrival_path = "./data_cleaned_capped/arrival_departure.parquet"

# laoding
survey_df = pd.read_csv(survey_path)
ridership_df = pd.read_csv(ridership_path)
arrival_departure_df = pd.read_parquet(arrival_path)

# print shapes 
print("survey_df:", survey_df.shape)
print("ridership_df:", ridership_df.shape)
print("arrival_departure_df:", arrival_departure_df.shape)


display(survey_df.head())
display(ridership_df.head())
display(arrival_departure_df.head())

/var/folders/zs/0gklbd017c758v1gqchfcyk00000gn/T/ipykernel_7637/119371211.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  ridership_df = pd.read_csv(ridership_path)


survey_df: (2335, 9)
ridership_df: (3327451, 17)
arrival_departure_df: (12682058, 14)


,aggregation_level,service_mode,reporting_group,measure_group,measure,category,weighted_percent,ObjectId,route_id
0,Reporting Group,Bus,10,Alternative Modes,Alternative Mode,Drive or Ride in a Carpool,0.316855,1,['10']
1,Reporting Group,Bus,10,Alternative Modes,Alternative Mode,"Bike, Scooter, or Other Micromobility",0.000000,2,['10']
2,Reporting Group,Bus,10,Alternative Modes,Alternative Mode,Private Shuttle or Other Transit,0.000000,3,['10']
3,Reporting Group,Bus,10,Alternative Modes,Alternative Mode,Other,0.000000,4,['10']
4,Reporting Group,Bus,10,Alternative Modes,Used Alternative Mode,No,0.837547,5,['10']


,alightings,boardings,day_type_id,day_type_name,direction_id,load_,route_id,route_variant,sample_size,season,stop_id,stop_name,stop_sequence,trip_start_time,source_file,term_from_name,year_from_name
0,0.3,0.4,day_type_01,weekday,0,7.8,1,1-0-0,13,Fall 2016,1,WASHINGTON ST OPP RUGGLES ST,2,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
1,0.0,0.1,day_type_01,weekday,0,9.4,1,1-0-0,13,Fall 2016,10003,ALBANY ST OPP RANDALL ST,5,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
2,0.4,0.0,day_type_01,weekday,0,10.4,1,1-0-0,13,Fall 2016,101,MASSACHUSETTS AVE @ SIDNEY ST,19,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
3,2.5,0.6,day_type_01,weekday,0,8.5,1,1-0-0,13,Fall 2016,102,MASSACHUSETTS AVE @ PROSPECT,20,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
4,0.2,0.1,day_type_01,weekday,0,8.4,1,1-0-0,13,Fall 2016,104,MASSACHUSETTS AVE @ BIGELOW S,21,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016


,service_date,route_id,stop_id,half_trip_id,time_point_id,time_point_order,actual,scheduled,scheduled_headway,headway,point_type,standard_type,earliness,direction
0,2018-08-01T00:00:00Z,1,75,40121394.0,mit,4.0,1900-01-01T05:19:34Z,1900-01-01T05:19:00Z,NaN,NaN,Midpoint,Schedule,-10.0,inbound
1,2018-08-01T00:00:00Z,1,79,40121394.0,hynes,5.0,1900-01-01T05:23:20Z,1900-01-01T05:22:00Z,NaN,NaN,Midpoint,Schedule,3.0,inbound
2,2018-08-01T00:00:00Z,1,187,40121394.0,masta,6.0,1900-01-01T05:25:58Z,1900-01-01T05:25:00Z,NaN,NaN,Midpoint,Schedule,-33.0,inbound
3,2018-08-01T00:00:00Z,1,59,40121394.0,Wasma,7.0,1900-01-01T05:28:26Z,1900-01-01T05:28:00Z,NaN,NaN,Midpoint,Schedule,7.0,inbound
4,2018-08-01T00:00:00Z,1,110,40121565.0,hhgat,1.0,1900-01-01T05:29:57Z,1900-01-01T05:30:00Z,1200.0,1218.0,Startpoint,Headway,NaN,inbound


### Inspect and Identify Feature Columns

In [3]:
# Overview of all three datasets 

def quick_overview(df, name, sample_cols=8):
    print(f"\n{'='*40}\n{name.upper()} — shape: {df.shape}\n{'='*40}")
    display(df.head(3))
    print("\nColumn summary:")
    display(df.dtypes)
    print("\nMissing values per column:")
    display(df.isna().sum().sort_values(ascending=False).head(sample_cols))
    print("\nUnique route_id examples:")
    if "route_id" in df.columns:
        print(df["route_id"].dropna().unique()[:20])
    elif "reporting_group" in df.columns:
        print(df["reporting_group"].dropna().unique()[:20])

quick_overview(ridership_df, "ridership_df")
quick_overview(arrival_departure_df, "arrival_departure_df")
quick_overview(survey_df, "survey_df")



RIDERSHIP_DF — shape: (3327451, 17)


,alightings,boardings,day_type_id,day_type_name,direction_id,load_,route_id,route_variant,sample_size,season,stop_id,stop_name,stop_sequence,trip_start_time,source_file,term_from_name,year_from_name
0,0.3,0.4,day_type_01,weekday,0,7.8,1,1-0-0,13,Fall 2016,1,WASHINGTON ST OPP RUGGLES ST,2,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
1,0.0,0.1,day_type_01,weekday,0,9.4,1,1-0-0,13,Fall 2016,10003,ALBANY ST OPP RANDALL ST,5,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
2,0.4,0.0,day_type_01,weekday,0,10.4,1,1-0-0,13,Fall 2016,101,MASSACHUSETTS AVE @ SIDNEY ST,19,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016



Column summary:


alightings         float64
boardings          float64
day_type_id         object
day_type_name       object
direction_id         int64
load_              float64
route_id            object
route_variant       object
sample_size          int64
season              object
stop_id              int64
stop_name           object
stop_sequence        int64
trip_start_time     object
source_file         object
term_from_name      object
year_from_name       int64
dtype: object


Missing values per column:


...


Column summary:


aggregation_level     object
service_mode          object
reporting_group       object
measure_group         object
measure               object
category              object
weighted_percent     float64
ObjectId               int64
route_id              object
dtype: object


Missing values per column:


aggregation_level    0
service_mode         0
reporting_group      0
measure_group        0
measure              0
category             0
weighted_percent     0
ObjectId             0
dtype: int64


Unique route_id examples:
["['10']" "['1']" "['104']" "['111']" "['11']" "['21']"
 "['114', '116', '117']" "['15']" "['16']" "['22']" "['28', '29']"
 "['23']" "['32']" "['34', '34E']" "['39']" "['43', 'SL5']" "['4', '7']"
 "['47']" "['57']" "['66']"]


### Aggregate ridership_df per route ID

In [4]:
rider_grp = ridership_df.groupby("route_id", dropna=True)

ridership_feat = pd.DataFrame({
    "boardings_mean": rider_grp["boardings"].mean(),
    "alightings_mean": rider_grp["alightings"].mean(),
    "load_mean": rider_grp["load_"].mean(),
    "boardings_median": rider_grp["boardings"].median(),
    "samples": rider_grp.size()
}).reset_index()

if "direction_id" in ridership_df.columns:
    dir_counts = (
        ridership_df.groupby(["route_id","direction_id"])
        .size()
        .unstack(fill_value=0)
        .rename(columns=lambda c: f"dir_{c}_count")
        .reset_index()
    )
    ridership_feat = ridership_feat.merge(dir_counts, on="route_id", how="left")

print("ridership_feat:", ridership_feat.shape)
display(ridership_feat.head())


ridership_feat: (58, 8)


,route_id,boardings_mean,alightings_mean,load_mean,boardings_median,samples,dir_0_count,dir_1_count
0,1,1.558645,1.558170,13.601183,0.7,141425,70145,71280
1,4,0.663981,0.713270,4.991266,0.1,1477,828,649
2,7,1.124647,1.117405,7.958976,0.1,30543,16125,14418
3,8,0.644464,0.646314,8.332684,0.2,72657,36867,35790
4,9,0.826970,0.823809,8.687067,0.2,93799,46515,47284


### Aggregate arrival_departure_df per route ID

In [5]:
arr = arrival_departure_df.copy()
arr_grp = arr.groupby("route_id", dropna=True)

arrival_feat = pd.DataFrame({
    "headway_mean": arr_grp["headway"].mean(),
    "headway_std": arr_grp["headway"].std(),
    "scheduled_headway_mean": arr_grp["scheduled_headway"].mean(),
    "earliness_mean": arr_grp["earliness"].mean(),
    "stops_count": arr_grp["stop_id"].nunique(),
    "records": arr_grp.size()
}).reset_index()

if "earliness" in arr.columns:
    arr["on_time"] = arr["earliness"].abs() <= 60
    on_time = (
        arr.groupby("route_id")["on_time"]
           .mean()
           .rename("on_time_rate")
           .reset_index()
    )
    arrival_feat = arrival_feat.merge(on_time, on="route_id", how="left")

print("arrival_feat:", arrival_feat.shape)
display(arrival_feat.head())


arrival_feat: (34, 8)


,route_id,headway_mean,headway_std,scheduled_headway_mean,earliness_mean,stops_count,records,on_time_rate
0,1,737.632266,469.237516,727.774157,-51.313495,16,642922,0.003173
1,10,784.192295,410.586677,709.524388,-165.253971,20,209898,0.221508
2,104,828.517231,418.517538,814.017844,-376.184918,12,182186,0.064121
3,11,658.554098,354.260679,653.832547,-167.481935,12,294675,0.122474
4,111,534.235697,398.856199,530.490678,-30.062437,21,1035012,0.002232


### Merge to form unified route-level feature table

In [6]:
route_feat = (
    ridership_feat.merge(arrival_feat, on="route_id", how="inner")
                  .dropna(axis=1, how="all")
)

print("route_feat:", route_feat.shape)
display(route_feat.head())


route_feat: (25, 15)


,route_id,boardings_mean,alightings_mean,load_mean,boardings_median,samples,dir_0_count,dir_1_count,headway_mean,headway_std,scheduled_headway_mean,earliness_mean,stops_count,records,on_time_rate
0,116,1.488029,1.495110,16.521192,0.5,1963,811,1152,1391.977672,725.387931,1372.253298,-35.093486,10,220549,0.007110
1,117,1.393963,1.395319,14.935521,0.5,2286,1083,1203,1689.356480,3804.558273,1682.007552,-18.507893,18,253382,0.007929
2,15,0.697025,0.698322,9.248568,0.3,4470,1943,2527,1008.157806,1698.923312,1001.001042,-73.313435,19,400914,0.004470
3,16,1.295473,1.294819,13.487525,0.5,1988,924,1064,766.331230,437.227041,699.657228,-177.959246,21,256754,0.170712
4,21,1.136508,1.166314,13.107231,0.3,567,273,294,752.669158,402.917429,745.791970,-196.103649,19,207241,0.125424


### Selecting features & scaling them

In [7]:
from sklearn.preprocessing import StandardScaler
import numpy as np

num_cols = route_feat.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric feature columns:", num_cols)

#  scale features 
X = route_feat[num_cols].fillna(route_feat[num_cols].median())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Numeric feature columns: ['boardings_mean', 'alightings_mean', 'load_mean', 'boardings_median', 'samples', 'dir_0_count', 'dir_1_count', 'headway_mean', 'headway_std', 'scheduled_headway_mean', 'earliness_mean', 'stops_count', 'records', 'on_time_rate']


#### Saving feture table

In [10]:
os.makedirs("./data_route_features", exist_ok=True)
feature_path = "./data_route_features/route_features.csv"

scaled_df = pd.DataFrame(X_scaled, columns=[f"{c}_scaled" for c in num_cols])
route_feat_save = pd.concat([route_feat.reset_index(drop=True), scaled_df], axis=1)

route_feat_save.to_csv(feature_path, index=False)
print(f" Saved route-level feature table to: {feature_path}")
print("Shape:", route_feat_save.shape)


 Saved route-level feature table to: ./data_route_features/route_features.csv
Shape: (25, 29)
